# 7 ドキュメント型DB（MongoDB）

- **[7.5 Pythonドライバ](#7.6-Pythonドライバ)**
    - [7.7.1 PyMongoのインストール](#7.7.1-PyMongoのインストール)
    - [7.7.2 PyMongoの使い方](#7.7.2-PyMongoの使い方)
<br><br>
- **[7.6 総合問題](#7.6-総合問題)**
    - [7.6.1 基本操作の復習](#7.6.1-基本操作の復習)
    - [7.6.2 Geospatialインデックスの応用](#7.6.2-Geospatialインデックスの応用)

***

## 7.5 Python ドライバ

本章の前半で述べた用に、PythonからMongoDBを使う場合は公式Pythonドライバ[PyMongo](https://api.mongodb.com/python/current/)を使います。こちら（第7章後半）では、前半と違って、Jupyterの**IPythonカーネル**を使用するので、コードセルでは普通のPythonコードおよびIPythonのマジックコマンド(`!<bash command>`, `%time`等）が使えます。

### 7.7.1 PyMongoのインストール

Jupyter環境上ではすでにインストールされていますが、自分のマシン等でPyMongoを使いたい場合は`pip`からインストールしてください:

```
$ pip install pymongo
```

また、`pymongo`本体以外に、高度な処理の場合以外は使う必要がないと思いますが、直接BSONオブジェクトを扱いたい場合は[`bson`](https://api.mongodb.com/python/3.4.0/api/bson/index.html)ライブラリ、GridFSを使いたいとき用の[`gridfs`](https://api.mongodb.com/python/3.4.0/api/gridfs/index.html)ライブラリもあります。それらのライブラリはPyMongoと一緒にインストールされるので、別途インストールする必要がありません。

### 7.7.2 PyMongoの使い方

PyMongoを使うにはまずはMongoDBと接続する`MongoClient`オブジェクトを生成します。

In [19]:
import pymongo
client = pymongo.MongoClient(host='localhost', port=27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

こちらでは、MongoDBのデフォルトホスト・ポートに接続しましたが、違うものを指定することで、外部のMongoDBインスタンス等と接続できます。詳しく知りたい読者は以下のセルを実行してdocstringを見てください。

In [3]:
?pymongo.MongoClient

まず、`client`オブジェクトを使ってデータベースを選択します。

In [20]:
db = client.get_database('twitter')

ここでは`get_database`を使いましたが、`client.twitter`でも同じです。また、データベース一覧を見たい場合（シェルでいう`show databases`）は、 `client.database_names()`を使います。

`tweets`コレクションを選択するときも、データベースと同様`db.tweets`もしくは`get_collection('tweets')`の2つのやり方があります。

それでは、早速データを取得してみましょう。

In [21]:
cursor = db.tweets.find({}, {'_id': 0, 'text': 1}).limit(10)
cursor

シェルと違って、pymongoの検索結果は**必ず**カーソルが返されます。実際の結果を表示するにはカーソルに何らかの操作を加えなければなりません。最もシンプルなのは単純に`list()`を呼ぶことです。

In [22]:
type(cursor)

pymongo.cursor.Cursor

In [23]:
list(cursor)

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

カーソルは一回しかiterateできないので、また`list()`を呼び出そうとすると、空のリストしか返ってこないです。

In [ ]:
list(cursor)

単純に`list()`を呼び出すだけだなく、基本的にiterableなオブジェクトにできるものは何でもできます。ただし、ジェネレーターと同じように、最後までiterateできるのは一回だけです。たとえば、先程の結果をより見やすくするために、以下のこともできます:

In [ ]:
cursor = db.tweets.find({}, {'_id': 0, 'text': 1}).limit(10)
[tweet['text'][:50] for tweet in cursor]

集計操作も基本的にシェルと同じです:

In [ ]:
cursor = db.tweets.aggregate([{"$match": {"lang": "en"}},
                     {"$project": {"num_hashtags": {"$size": "$hashtags"}}},
                     {"$group": {"_id": "$num_hashtags", "count" : {"$sum" : 1}}},
                     {"$sort": {"_id": 1}}
                    ])
list(cursor)

しかし、挿入・更新・削除等の[`camelCase`](https://ja.wikipedia.org/wiki/キャメルケース)を使うコマンド名はすべて[`snake_case`](https://en.wikipedia.org/wiki/Snake_case)になります。以下の表で確認してください。

<p id="table10" style="text-align:center">**表10** Mongoシェル/Pymongoコマンド対照表</p>

| Mongo Shell | Pymongo |
| :-----: |:-----------:|
|`insertOne` | `insert_one`|
|`insertMany` | `insert_many`|
|`updateOne` | `update_one`|
|`updateMany` | `update_many`|
|`deleteOne` | `delete_one`|
|`deleteMany` | `delete_many`|

また、Mongoシェルではフィールド名や`$`で始まる演算子はそのまま（引用符なし）で使えますが、先程の例にあるように、Pythonでは明示的に引用符を使って文字列にしなければなりません。

その他にいくつかの微妙な違いがあるのですが、必要に応じてPyMongoの[APIドキュメンテーション](http://api.mongodb.com/python/current/api/pymongo/index.html)を見てください。

***

## 7.6 総合問題

### 7.6.1 基本操作の復習

1. 各ツイートのユーザ位置情報に「Japan」もしくは「日本」という文字列が含まれていてかつ言語が英語であるものを検索し、該当ツイートの本文および`ObjectId`のみを取得せよ。
2. 各ツイートのユーザ位置情報を言語毎の集合にグループ化せよ。（ヒント: `$project`、`$unwind`、`$addToSet`を使用せよ。）

### 7.6.2 Geospatialインデックスの応用

これまで使ってきたTwitterデータののほとんどには位置情報が付いています。その位置情報は文字列ですが、[Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/start)等を使って国・都市の名前から緯度・経度を取得することが可能です。(1)その緯度・経度情報を取得し、(2)各ドキュメントに新しいフィールドとして緯度・経度情報を追加した上で、(3)そのフィールドに対してGeospatialインデックスの一種である[2dsphere](https://docs.mongodb.com/manual/core/2dsphere/)を作ってください。その上で、(4)[`$near`](https://docs.mongodb.com/manual/reference/operator/query/near/)演算子を使って東京から5000km以上15000km未満の距離にあるユーザのツイートを取得せよ。

- **注1**: 位置データが無効もしくは存在しないツイートは無視してください。
- **注2**: APIコールには[`requests`](http://docs.python-requests.org/en/master/)、API結果の処理には[`json`](https://docs.python.org/3/library/json.html)が便利でしょう。